This test is invalid when the observed or expected frequencies in each category are too small.
A typical rule is that all of the observed and expected frequencies should be at least 5.
According to [3], the total number of samples is recommended to be greater than 13, otherwise exact tests (such as Barnard’s Exact test) should be used because they do not overreject.

Also, the sum of the observed and expected frequencies must be the same for the test to be valid; chisquare raises an error if the sums do not agree within a relative tolerance of 1e-8.

The default degrees of freedom, k-1, are for the case when no parameters of the distribution are estimated. If p parameters are estimated by efficient maximum likelihood then the correct degrees of freedom are k-1-p.
If the parameters are estimated in a different way, then the dof can be between k-1-p and k-1.
However, it is also possible that the asymptotic distribution is not chi-square, in which case this test is not appropriate.

In [22]:
import bundle.baselines.st3 as st3
import bundle.baselines.st2 as st2
from models.helpers import *
import pandas as pd
from tqdm import tqdm
import os
import sys
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.metrics import classification_report as report
from sklearn.feature_extraction.text import CountVectorizer
import argparse
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer


In [9]:
languages_with_train = ["en", "fr", "ge", "it", "po", "ru"]
languages_without_train = ["es", "gr", "ka"]

paths_st3 = get_paths('en')
paths_st2 = get_paths('en', subtask=2)

st3_train = st3.make_dataframe(paths_st3["train_folder"], paths_st3["train_labels"])

st3_test = st3.make_dataframe(paths_st3["dev_folder"], paths_st3["dev_labels"])

st2_train = st2.make_dataframe(paths_st2["train_folder"], paths_st2["train_labels"])
    
st2_test = st2.make_dataframe(paths_st2["dev_folder"], paths_st2["dev_labels"])

446it [00:00, 16284.59it/s]
90it [00:00, 23917.34it/s]
433it [00:00, 43225.84it/s]
83it [00:00, 39225.60it/s]


In [28]:
st3_train.reset_index()
st3_train.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3760 entries, (111111111, 3) to (999001970, 13)
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3760 non-null   object
 1   labels  3760 non-null   object
dtypes: object(2)
memory usage: 223.6+ KB


In [26]:
st2_train.reset_index()

,id,text,frames
0,833042063,Chelsea Handler Admits She’s ‘Very Sexually At...,"Political,Morality,Fairness_and_equality,Exter..."
1,832959523,How Theresa May Botched\n\nThose were the time...,"Morality,Security_and_defense,Policy_prescript..."
2,833039623,Robert Mueller III Rests His Case—Dems NEVER W...,"Political,Crime_and_punishment,External_regula..."
3,833032367,Robert Mueller Not Recommending Any More Indic...,"Political,Crime_and_punishment,Fairness_and_eq..."
4,814777937,The Far Right Is Trying to Co-opt the Yellow V...,"Political,Morality,Fairness_and_equality,Exter..."
...,...,...,...
428,832908978,Trump Kept Asking Friends If the Mueller Repor...,"Political,Crime_and_punishment,External_regula..."
429,832910505,"The Mueller Circus is Over, Long Live the Muel...","Political,Morality,Crime_and_punishment,Qualit..."
430,832917532,Robert Mueller Not Recommending Any More Indic...,"Crime_and_punishment,Political,Legality_Consti..."
431,832913653,Kamala Harris calls for AG Barr to testify on ...,"Political,Crime_and_punishment,Fairness_and_eq..."


In [27]:
# I want to join the two dataframes on the "id" column
# use merge with inner join

train = pd.merge(st3_train, st2_train, on="id", how="inner")
train

ValueError: You are trying to merge on int64 and object columns for key 'id'. If you wish to proceed you should use pd.concat